In [3]:
'''
This class defines the object "virtual sensor" that should hold 
(i) it's position (or coordinates) in a 2D space
(ii) a covariance matrix (or variance value) that corresponds to an *assumed* 
distance dependent error around a 2D sphere having the sensor as it's origin
and the actual (or distorted) distance to the ship as radius

CURRENT VERSION: The class only hold coordinates, a METHOD that calculates
the sensor's distance to the ship (and saves it as self.r), and another one
that uses self.r as input and saves the assumed stand. deriv. of the spherical
error around the circle through the ship as self.sig. Furthermore, it holds a 
METHOD that (based on self.coord, self.r, self.sig) uses a point (x,y) as input
to which a "pseudo-likelihood" output in (0,1] is produced according to the
following procedure:
    - Project the point onto the closest point (x_circ, y_circ) of the sphere
    around the sensor (with self.coord position) with radius self.r
        - Through polar coordinates
    - Define the likelihood value of the point to be the value of the normal
    distribution density with mean (x_circ, y_circ) and stdev. self.sig
       
NEXT VERSION: (If it makes sense) Implement another standard deviation value
that encapsulates prediction errors orthogonal to direct paths towards the ship
and use it as the second component to use the multivariate normal distribution
density (with the resulting (co-)variance matrix) as likelihood
 
LATER VERSION: Integrate the class into a class "sensor network" that holds
the position of m sensors in a 2D space, and automatically multiplies all 
likelihood functions of the individual sensors for a given point (which is
currently done separately)
'''

import numpy.linalg as linalg
import numpy as np
import scipy.stats as stats
import numpy.random as rnd
import numpy.matlib as matlib

class virt_sensor():
    def __init__(self, coord):
        self.coord = coord
            
    def def_gaussian_sphere_vals(self, ship, err_mean, err_stdev): # We assume
    # "ship" to be a class having it's position saved as .coord.
    # The error "err_mean" is the multiplicative value k in
    # k*log(true_distance), and is used to define the standard deviation of a random 
    # draw from N(0, k*log(true_distance)) that is added to the true_distance of a
    # sensor to the ship and defines the distorted distance "dist"
    # The error "err_stdev" (non-zero real value) is used as multiplicative
    # factor k in k*log(distorted_distance) which defines the standard deviation
    # assumed for the spherical normal distribution around the sphere (as mean)
    # with radius "dist" (distorted distance) around the sensor's position
        dist = linalg.norm(self.coord - ship.coord) # True Euclidean distance to ship
        # Since the standard deviation cannot be 0 we assume the errors to be
        # proportional to log(1+dist)
        self.r = dist + rnd.normal(0, err_mean*np.log(1 + dist)) # Save the 
        # *distorted* radius of the sphere
        self.stdev = err_stdev * np.log(1 + self.r) # Save the distance dependent 
        # *assumed* standard deviation orthogonal to any tangent on the circle
        # around self.coord with radius self.r (CURRENT VERSION: Based on 
        # propagation equation the error is assumed to be logarithmic in the 
        # distance)
        
        # In addition to the distorted radii and stdev of the Gaussian sphere,
        # we will weight the implied likelihood predictions invertedly to
        # the distorted distance to the ship, i.e. we will assign the prediction
        # of the closest sensor highest within the likelihood. For this matter
        # we assign every sensor it's individual prediction weight w_i, and
        # weight it's likelihood prediction with w_i/w where w is the sum of
        # all individual weight (which is CURRENTLY assigned in the class of 
        # the sensor network)
        self.pred_weight = (1 + self.r)**(-1) # +1 to ensure that +inf is not possible
    
    def get_likelihood(self, xy): # At this point the self values ".r" and 
    # ".stdev" are assumed to be defined. This method then takes a 2D point xy
    # calculates the position xy_circ of its closest point on the sphere with
    # radius self.r around self.coord (via polar coordinates), and returns
    # the N(xy_circ, self.stdev) density value of the point xy
        xy_rel = xy - self.coord # Cartesian coordinates relative to self.coord
        r_rel = linalg.norm(xy_rel) # Distance relative to self.coord
        if (r_rel == 0): # If the point is directly on top of the sensor
        # the polar coordinates are not defined (CURRENT VERSION: Declare the
        # likelihood at this point to be 0)
            return 0
        
        if(xy_rel[1]>=0) & (r_rel != 0): # Calculate the angle of xy relative
        # to self.coord via polar coordinate formulae +-arccos(x/r)
            phi_rel = np.arccos(xy_rel[0]/r_rel)
        elif(xy_rel[1]<0) & (r_rel != 0):
            phi_rel = - np.arccos(xy_rel[0]/r_rel)
        else:
            print("Cannot calculate the likelihood at " + str(xy) + " since "+
                  "the radius is 0.")
            return 
        # Define the closest point on the sphere as the Cartesian point with
        # polar coordinate radius self.r but for the same angle (phi_rel) 
        # relative to self.coord 
        # NOTE: This is relative to the sensor position such that we have to
        # shift it by +self.coord
        xy_circ = np.array([self.r * np.cos(phi_rel), self.r * np.sin(phi_rel)]) + self.coord
        val = stats.norm.pdf(linalg.norm(xy-xy_circ), loc = 0, scale = self.stdev) # Calculate
        # probability density value at xy for N(xy_circ, self.stdev) distribution
        return val
            
    

class virt_sensor_net():
    def __init__(self, num_sens, coords_vec):
        self.size = num_sens
        self.sensors = [] 
        for i in range(num_sens):
            self.sensors.append(virt_sensor(coords_vec[i]))
    
    def def_gaussian_spheres(self, ship, err_mean, err_stdev):
        # Following weight will be used in the likelihood prediction as normalisation
        # constant
        pred_weight = 0
        for i in range(self.size):
            self.sensors[i].def_gaussian_sphere_vals(ship, err_mean, err_stdev)
            pred_weight = self.sensors[i].pred_weight
            self.total_pred_weight = pred_weight
            
    def get_net_likelihood(self, xy):
        # Following is for multiplicatively weighted predictions
        
        val = 1
        
        # ALTERNATIVELY for log likelihood approach (resulting in sums)
        '''
        val = 0
        '''
        for i in range(self.size):
            # Weight each individual prediction by w_i according to the above
            # rules (i.e. invertedly to the distance to the ship) without 
            # normalisation in the multiplicative case (for normal likelihood)
            
            ind_val = self.sensors[i].get_likelihood(xy)
            weight = self.sensors[i].pred_weight
            val = val * weight * ind_val
            
            # ALTERNATIVELY: log likelihood with weights w_i/w
            '''
            ind_val = np.log(self.sensors[i].get_likelihood(xy))
            weight = self.sensors[i].pred_weight/self.total_pred_weight
            val = val + weight * ind_val
            '''
            
        return val
            

# NEW sensor network class in which the network will be defined entirely and
# not through individual sensors. It also assumes the ship fleet class to be 
# defined equivalently
class virt_sensor_net2():
    def __init__(self, coords_vec):
        self.size = coords_vec.shape[0]
        self.coords = coords_vec 
    
    def def_gaussian_spheres(self, ship_fleet, err_mean, err_stdev):
        # TODO: Double check if this actually does what it should
        # Goal: Define a matrix "dist" where dist[i] is a (row) vector of m
        # values holding the distances of each of m vectors to ship i
        # (i) Repeat each row of the nx2 matrix ship_fleet.coord m times and
        # reshape it in 3 dimensions such that (0,:,:) holds a matrix of the
        # ship_fleet.coord[0] row repeated m times vertically, and (n,:,:) does
        # the same for ship_fleet.coord[-1]
        fleet_coords_3d = matlib.repmat(ship_fleet.coords, 1, self.size).reshape((ship_fleet.size, self.size, 2))
        # (ii) Substracting with the mx2 self.coords matrix results in the
        # entry [k,i,:] to contain the row vector of the difference of
        # ship_fleet.coord[k] and sensor[i]. Then taking the linalg.norm of
        # the difference w.r.t. axis = 2 ends up in an nxm matrix where entry
        # [i,j] contains the Euclidean distance of sensor j to ship i (as wanted)
        dist = linalg.norm(fleet_coords_3d - self.coords, axis = 2)
        # Addint a centralised normally distributed error where each stdev
        # is defined by err_mean*log(1 + dist) and save these distorted radii
        # of the Gaussian spheres
        self.radii = dist + rnd.normal(0, err_mean * np.log(1 + dist)) # TODO: 
        # Double check if this adds the correct errors to the correct 
        # sensor-ship-distances
        self.stdevs = err_stdev * np.log(1 + self.radii) # TODO: Double check
        # the same here AND take description from above
            
    def get_net_likelihood(self, ship_index, xy):
        xy_rel = xy - self.coords # mx2 vector
        r_rel = linalg.norm(xy_rel, axis = 1) #mx1 vector
        
        # CURRENT VERSION: If xy point is on top of one of the sensors but the
        # distorted distance to the ship is not 0, declare the likelihood as 0
        # and if it is on top and the distance is 0, declare it as 1
        ''' DOES NOT WORK CURRENTLY
        zero_idx = (r_rel == 0)
        zero_idx_size = sum(zero_idx.astype(int))
        if(self.radii[ship_index, zero_idx] != np.zeros((1,zero_idx_size))):
            return 0
        elif(self.radii[ship_index, zero_idx] == np.zeros((1,zero_idx_size))):
            return 1
        '''
        
        # Calculate the polar coordinates and return the mx2 matrix of sphere
        # projection coordinates of xy onto the m Gaussian spheres
        phis = np.zeros(self.size) # Polar coordinate angle relative to self.coords
        idx1 = (r_rel != 0) & (xy_rel[:,1]>=0) # Those indices get arccos(x/r)
        idx2 = (r_rel != 0) & (xy_rel[:,1]<0) # Those indices get -arccos(x/r)
        phis[idx1] = np.arccos(xy_rel[idx1,0]/r_rel[idx1])
        phis[idx2] = - np.arccos(xy_rel[idx2,0]/r_rel[idx2])
        
        x_proj = self.radii[ship_index] * np.cos(phis)
        y_proj = self.radii[ship_index] * np.sin(phis)
        xy_proj = self.coords + np.hstack((x_proj.reshape(-1,1), y_proj.reshape(-1,1)))
        
        # TODO: Double check if the following does exactly what it should
        lhd_val_vec = stats.norm.pdf(linalg.norm(xy-xy_proj), loc = 0, 
                                     scale = self.stdevs[ship_index])
        # CURRENT VERSION: Returns the weighted likelihood product
        weights = (1 + self.radii[ship_index])**(-1)
        return np.prod(weights * lhd_val_vec)
        
    
# NEW NEW sensor network class in which the definition of the Gaussian spheres
# can be omitted since the likelihood value only depends on the value
# || x - sensor_pos || - || measured_ship_pos - sensor_pos ||
class virt_sensor_net_v3():
    def __init__(self, coords_vec):
        self.size = coords_vec.shape[0]
        self.coords = coords_vec 
    
    def def_gaussian_spheres(self, ship_fleet, err_mean, err_stdev):
        # TODO: Double check if this actually does what it should
        # Goal: Define a matrix "dist" where dist[i] is a (row) vector of m
        # values holding the distances of each of m vectors to ship i
        # (i) Repeat each row of the nx2 matrix ship_fleet.coord m times and
        # reshape it in 3 dimensions such that (0,:,:) holds a matrix of the
        # ship_fleet.coord[0] row repeated m times vertically, and (n,:,:) does
        # the same for ship_fleet.coord[-1]
        fleet_coords_3d = matlib.repmat(ship_fleet.coords, 1, self.size).reshape((ship_fleet.size, self.size, 2))
        # (ii) Substracting with the mx2 self.coords matrix results in the
        # entry [k,i,:] to contain the row vector of the difference of
        # ship_fleet.coord[k] and sensor[i]. Then taking the linalg.norm of
        # the difference w.r.t. axis = 2 ends up in an nxm matrix where entry
        # [i,j] contains the Euclidean distance of sensor j to ship i (as wanted)
        dist = linalg.norm(fleet_coords_3d - self.coords, axis = 2)
        # Adding a centralised normally distributed error where each stdev
        # is defined by err_mean*log(1 + dist) and save these distorted radii
        # of the Gaussian spheres
        self.radii = dist + rnd.normal(0, err_mean * np.log(1 + dist)) # TODO: 
        # Double check if this adds the correct errors to the correct 
        # sensor-ship-distances
        self.stdevs = err_stdev * np.log(1 + self.radii) # TODO: Double check
        # the same here AND take description from above
            
    def get_net_likelihood(self, ship_index, xy):
        xy_rel = xy - self.coords # mx2 vector
        r_rel = linalg.norm(xy_rel, axis = 1) #mx1 vector
        
        # CURRENT VERSION: If xy point is on top of one of the sensors but the
        # distorted distance to the ship is not 0, declare the likelihood as 0
        # and if it is on top and the distance is 0, declare it as 1
        ''' DOES NOT WORK CURRENTLY
        zero_idx = (r_rel == 0)
        zero_idx_size = sum(zero_idx.astype(int))
        if(self.radii[ship_index, zero_idx] != np.zeros((1,zero_idx_size))):
            return 0
        elif(self.radii[ship_index, zero_idx] == np.zeros((1,zero_idx_size))):
            return 1
        '''
        
        # Calculate the polar coordinates and return the mx2 matrix of sphere
        # projection coordinates of xy onto the m Gaussian spheres
        phis = np.zeros(self.size) # Polar coordinate angle relative to self.coords
        idx1 = (r_rel != 0) & (xy_rel[:,1]>=0) # Those indices get arccos(x/r)
        idx2 = (r_rel != 0) & (xy_rel[:,1]<0) # Those indices get -arccos(x/r)
        phis[idx1] = np.arccos(xy_rel[idx1,0]/r_rel[idx1])
        phis[idx2] = - np.arccos(xy_rel[idx2,0]/r_rel[idx2])
        
        x_proj = self.radii[ship_index] * np.cos(phis)
        y_proj = self.radii[ship_index] * np.sin(phis)
        xy_proj = self.coords + np.hstack((x_proj.reshape(-1,1), y_proj.reshape(-1,1)))
        
        # TODO: Double check if the following does exactly what it should
        lhd_val_vec = stats.norm.pdf(linalg.norm(xy-xy_proj), loc = 0, 
                                     scale = self.stdevs[ship_index])
        # CURRENT VERSION: Returns the weighted likelihood product
        weights = (1 + self.radii[ship_index])**(-1)
        return np.prod(weights * lhd_val_vec)